# **SET-UP**


In [1]:
import os
import pandas as pd

!rm -rf /content/fifa-scouting-end-to-end
!git clone https://github.com/ElouanBoropert/fifa-scouting-end-to-end.git

REPO_DIR = "/content/fifa-scouting-end-to-end"

candidates = [
    os.path.join(REPO_DIR, "data/raw/fifa_players.csv"),
    "/content/fifa_players.csv",
]

csv_path = next((p for p in candidates if os.path.exists(p)), None)

if csv_path is None:
    from google.colab import files
    uploaded = files.upload()  #fifa_players.csv
    csv_path = next(iter(uploaded.keys()))

print("CSV used:", csv_path)

df = pd.read_csv(csv_path)
print("Shape (lignes, colonnes):", df.shape)
df.head(3)


Cloning into 'fifa-scouting-end-to-end'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 17 (delta 2), reused 11 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 1.20 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (2/2), done.
CSV used: /content/fifa-scouting-end-to-end/data/raw/fifa_players.csv
Shape (lignes, colonnes): (17954, 51)


,name,full_name,birth_date,age,height_cm,weight_kgs,positions,nationality,overall_rating,potential,...,long_shots,aggression,interceptions,positioning,vision,penalties,composure,marking,standing_tackle,sliding_tackle
0,L. Messi,Lionel Andrés Messi Cuccittini,6/24/1987,31,170.18,72.1,"CF,RW,ST",Argentina,94,94,...,94,48,22,94,94,75,96,33,28,26
1,C. Eriksen,Christian Dannemann Eriksen,2/14/1992,27,154.94,76.2,"CAM,RM,CM",Denmark,88,89,...,89,46,56,84,91,67,88,59,57,22
2,P. Pogba,Paul Pogba,3/15/1993,25,190.50,83.9,"CM,CAM",France,88,91,...,82,78,64,82,88,82,87,63,67,67


# **Basic Analysis**

In [4]:
## Quick Checks

# Columns
print("Columns Number:", len(df.columns))
print(df.columns.tolist())

# Types + missing
display(df.dtypes.value_counts())
missing = df.isna().sum().sort_values(ascending=False)
display(missing.head(20))

# Duplicates
dup = df.duplicated(subset=["full_name", "birth_date"]).sum()
print("Duplicates (full_name + birth_date):", dup)

# Sanity Checks
print("overall_rating min/max:", df["overall_rating"].min(), df["overall_rating"].max())
print("potential min/max:", df["potential"].min(), df["potential"].max())
print("value_euro min/max:", df["value_euro"].min(), df["value_euro"].max())
print("wage_euro min/max:", df["wage_euro"].min(), df["wage_euro"].max())

# Positions: examples
display(df["positions"].head(10))
print("Number of Unique Positions (string):", df["positions"].nunique())


Columns Number: 51
['name', 'full_name', 'birth_date', 'age', 'height_cm', 'weight_kgs', 'positions', 'nationality', 'overall_rating', 'potential', 'value_euro', 'wage_euro', 'preferred_foot', 'international_reputation(1-5)', 'weak_foot(1-5)', 'skill_moves(1-5)', 'body_type', 'release_clause_euro', 'national_team', 'national_rating', 'national_team_position', 'national_jersey_number', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'freekick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'composure', 'marking', 'standing_tackle', 'sliding_tackle']


,count
int64,35
object,9
float64,7


,0
national_team,17097
national_team_position,17097
national_rating,17097
national_jersey_number,17097
release_clause_euro,1837
value_euro,255
wage_euro,246
positions,0
birth_date,0
name,0


Duplicates (full_name + birth_date): 0
overall_rating min/max: 47 94
potential min/max: 48 95
value_euro min/max: 10000.0 110500000.0
wage_euro min/max: 1000.0 565000.0


,positions
0,"CF,RW,ST"
1,"CAM,RM,CM"
2,"CM,CAM"
3,"LW,ST"
4,CB
5,CB
6,"RW,ST,RM"
7,ST
8,GK
9,ST


Number of Unique Positions (string): 890


# **Cleaning**

In [5]:
import os
import re
import numpy as np
import pandas as pd

# --- Paths ---
REPO_DIR = "/content/fifa-scouting-end-to-end"
RAW_PATH = os.path.join(REPO_DIR, "data/raw/fifa_players.csv")

# If raw file isn't in the repo path, fallback to previous csv_path variable if it exists
if not os.path.exists(RAW_PATH):
    try:
        RAW_PATH = csv_path  # from previous step
    except NameError:
        raise FileNotFoundError("Raw CSV not found. Please run Step 2 setup cell first.")

df_raw = pd.read_csv(RAW_PATH)

# --- Helper: safe numeric conversion ---
def to_numeric_safe(s: pd.Series) -> pd.Series:
    # Convert to numeric, forcing errors to NaN
    return pd.to_numeric(s, errors="coerce")

# --- 1) Standardize column names
df = df_raw.copy()
df.columns = [c.strip() for c in df.columns]

# --- 2) Fix positions: keep as a clean list and extract primary position ---
# Some datasets have positions as comma-separated values (e.g., "ST,CF,RW")
# I keep both raw string and derived columns for modeling later.
df["positions"] = df["positions"].astype(str).str.strip()

df["positions_list"] = (
    df["positions"]
    .str.replace(r"\s+", "", regex=True)        # remove spaces
    .str.split(",")                             # split into list
)

df["primary_position"] = df["positions_list"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else np.nan)
df["positions_count"] = df["positions_list"].apply(lambda x: len(x) if isinstance(x, list) else 0)

# --- 3) Parse birth_date and compute age (approx, based on today's date in Colab) ---
# birth_date example: "6/24/1987"
df["birth_date"] = pd.to_datetime(df["birth_date"], errors="coerce")

today = pd.Timestamp.today().normalize()
df["age"] = ((today - df["birth_date"]).dt.days / 365.25).round(1)

# --- 4) Ensure key numeric fields are numeric ---
num_cols = [
    "height_cm", "weight_kgs",
    "overall_rating", "potential",
    "value_euro", "wage_euro", "release_clause_euro",
]

for c in num_cols:
    if c in df.columns:
        df[c] = to_numeric_safe(df[c])

# --- 5) Basic sanity rules / flags (do not drop yet, just flag) ---
df["flag_missing_birth_date"] = df["birth_date"].isna()
df["flag_overall_gt_potential"] = (df["overall_rating"] > df["potential"]) & df["overall_rating"].notna() & df["potential"].notna()
df["flag_non_positive_value"] = (df["value_euro"] <= 0) & df["value_euro"].notna()
df["flag_non_positive_wage"] = (df["wage_euro"] <= 0) & df["wage_euro"].notna()

# --- 6) Remove exact duplicate players based on a simple key ---
if {"full_name", "birth_date"}.issubset(df.columns):
    before = len(df)
    df = df.drop_duplicates(subset=["full_name", "birth_date"], keep="first").copy()
    after = len(df)
else:
    before, after = len(df), len(df)

# --- 7) Create a stable surrogate key (player_key) ---
# Useful later for SQL modeling even if there is no unique id
def make_player_key(row) -> str:
    name = str(row.get("full_name", "")).strip().lower()
    bd = row.get("birth_date")
    bd_str = bd.strftime("%Y-%m-%d") if pd.notna(bd) else "unknown"
    return re.sub(r"[^a-z0-9]+", "-", f"{name}-{bd_str}").strip("-")

df["player_key"] = df.apply(make_player_key, axis=1)

# --- 8) Export processed outputs ---
processed_dir = os.path.join(REPO_DIR, "data/processed")
os.makedirs(processed_dir, exist_ok=True)

clean_csv_path = os.path.join(processed_dir, "players_clean.csv")
df.to_csv(clean_csv_path, index=False)

# Parquet (smaller + faster for later steps)
clean_parquet_path = os.path.join(processed_dir, "players_clean.parquet")
df.to_parquet(clean_parquet_path, index=False)

print("Saved:", clean_csv_path)
print("Saved:", clean_parquet_path)
print("Rows raw:", len(df_raw), "| Rows clean:", len(df))


Saved: /content/fifa-scouting-end-to-end/data/processed/players_clean.csv
Saved: /content/fifa-scouting-end-to-end/data/processed/players_clean.parquet
Rows raw: 17954 | Rows clean: 17954


# **Data Quality Report**

In [6]:
# --- Data quality summary ---
report = {}

report["rows_raw"] = int(len(df_raw))
report["rows_clean"] = int(len(df))
report["duplicates_removed"] = int(len(df_raw) - len(df))

# Missingness
missing = df.isna().mean().sort_values(ascending=False)
report["top_missing_cols"] = missing.head(15).round(3).to_dict()

# Flag rates
flag_cols = [c for c in df.columns if c.startswith("flag_")]
flag_rates = {c: float(df[c].mean()) for c in flag_cols}
report["flag_rates"] = {k: round(v, 3) for k, v in flag_rates.items()}

# Ranges (core metrics)
ranges = {}
for c in ["overall_rating", "potential", "value_euro", "wage_euro", "release_clause_euro", "age"]:
    if c in df.columns:
        ranges[c] = {
            "min": float(np.nanmin(df[c].values)) if df[c].notna().any() else None,
            "max": float(np.nanmax(df[c].values)) if df[c].notna().any() else None,
        }
report["ranges"] = ranges

report_md = []
report_md.append("# Data Quality Report\n")
report_md.append(f"- Rows raw: **{report['rows_raw']}**")
report_md.append(f"- Rows clean: **{report['rows_clean']}**")
report_md.append(f"- Duplicates removed (full_name + birth_date): **{report['duplicates_removed']}**\n")

report_md.append("## Top missing columns (rate)\n")
for k, v in report["top_missing_cols"].items():
    report_md.append(f"- {k}: {v}")

report_md.append("\n## Flag rates\n")
for k, v in report["flag_rates"].items():
    report_md.append(f"- {k}: {v}")

report_md.append("\n## Key ranges\n")
for k, d in report["ranges"].items():
    report_md.append(f"- {k}: min={d['min']}, max={d['max']}")

report_path = os.path.join(REPO_DIR, "docs", "DATA_QUALITY_REPORT.md")
os.makedirs(os.path.dirname(report_path), exist_ok=True)

with open(report_path, "w", encoding="utf-8") as f:
    f.write("\n".join(report_md))

print("Saved report:", report_path)


Saved report: /content/fifa-scouting-end-to-end/docs/DATA_QUALITY_REPORT.md


# **Export & GitHub artifacts**

In [8]:
import os

# Create a small sample that can be tracked on GitHub (lightweight)
sample_dir = os.path.join(REPO_DIR, "data", "sample")
os.makedirs(sample_dir, exist_ok=True)

sample_path = os.path.join(sample_dir, "players_clean_sample.csv")
df.sample(n=min(500, len(df)), random_state=42).to_csv(sample_path, index=False)

print("Saved sample:", sample_path)
!ls -lh /content/fifa-scouting-end-to-end/data/sample | head


Saved sample: /content/fifa-scouting-end-to-end/data/sample/players_clean_sample.csv
total 140K
-rw-r--r-- 1 root root 139K Jan 19 19:25 players_clean_sample.csv


In [9]:
import os
from textwrap import dedent

script_path = os.path.join(REPO_DIR, "src", "clean_players.py")
os.makedirs(os.path.dirname(script_path), exist_ok=True)

code = dedent("""
import re
import os
import numpy as np
import pandas as pd

def to_numeric_safe(s: pd.Series) -> pd.Series:
    # Convert to numeric, forcing errors to NaN
    return pd.to_numeric(s, errors="coerce")

def make_player_key(full_name: str, birth_date: pd.Timestamp) -> str:
    # Create a stable surrogate key from name + birth_date
    name = str(full_name).strip().lower()
    bd_str = birth_date.strftime("%Y-%m-%d") if pd.notna(birth_date) else "unknown"
    return re.sub(r"[^a-z0-9]+", "-", f"{name}-{bd_str}").strip("-")

def clean_players(input_csv: str, output_csv: str, output_parquet: str) -> pd.DataFrame:
    df_raw = pd.read_csv(input_csv)
    df = df_raw.copy()
    df.columns = [c.strip() for c in df.columns]

    # Positions cleanup
    df["positions"] = df["positions"].astype(str).str.strip()
    df["positions_list"] = (
        df["positions"]
        .str.replace(r"\\s+", "", regex=True)
        .str.split(",")
    )
    df["primary_position"] = df["positions_list"].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else np.nan)
    df["positions_count"] = df["positions_list"].apply(lambda x: len(x) if isinstance(x, list) else 0)

    # Parse birth_date + age
    df["birth_date"] = pd.to_datetime(df["birth_date"], errors="coerce")
    today = pd.Timestamp.today().normalize()
    df["age"] = ((today - df["birth_date"]).dt.days / 365.25).round(1)

    # Numeric fields
    num_cols = [
        "height_cm", "weight_kgs",
        "overall_rating", "potential",
        "value_euro", "wage_euro", "release_clause_euro",
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = to_numeric_safe(df[c])

    # Quality flags (kept as columns)
    df["flag_missing_birth_date"] = df["birth_date"].isna()
    df["flag_overall_gt_potential"] = (df["overall_rating"] > df["potential"]) & df["overall_rating"].notna() & df["potential"].notna()
    df["flag_non_positive_value"] = (df["value_euro"] <= 0) & df["value_euro"].notna()
    df["flag_non_positive_wage"] = (df["wage_euro"] <= 0) & df["wage_euro"].notna()

    # Drop duplicates
    if {"full_name", "birth_date"}.issubset(df.columns):
        df = df.drop_duplicates(subset=["full_name", "birth_date"], keep="first").copy()

    # Surrogate key
    df["player_key"] = df.apply(lambda r: make_player_key(r.get("full_name", ""), r.get("birth_date", pd.NaT)), axis=1)

    # Export
    os.makedirs(os.path.dirname(output_csv), exist_ok=True)
    df.to_csv(output_csv, index=False)
    df.to_parquet(output_parquet, index=False)

    return df

if __name__ == "__main__":
    # Example usage (adjust paths as needed)
    repo_dir = "/content/fifa-scouting-end-to-end"
    input_csv = os.path.join(repo_dir, "data", "raw", "fifa_players.csv")
    out_dir = os.path.join(repo_dir, "data", "processed")
    output_csv = os.path.join(out_dir, "players_clean.csv")
    output_parquet = os.path.join(out_dir, "players_clean.parquet")

    df_clean = clean_players(input_csv, output_csv, output_parquet)
    print("Done. Clean rows:", len(df_clean))
""").strip() + "\n"

with open(script_path, "w", encoding="utf-8") as f:
    f.write(code)

print("Saved script:", script_path)
!ls -lh /content/fifa-scouting-end-to-end/src | head

Saved script: /content/fifa-scouting-end-to-end/src/clean_players.py
total 4.0K
-rw-r--r-- 1 root root 3.0K Jan 19 19:27 clean_players.py
